# Language Classifier

### Setup und Helpers

In [ ]:
import numpy as np

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
import sys
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.evaluation import MulticlassMetrics

### Übung 1

Erstellen eines neuen SparkContext mit dem Name "LanguageClassifier"

In [ ]:
def printRDD(rdd, part = True):
    if (part):
        for i in rdd.take(10):
            print(i)
    else:
        for i in rdd.collect():
            print(i)  

### Übung 2

Die Methode "readDataFiles" ergänzen. Erstelle eine Variable mit dem Namen dataRDD und lade mit dem SparkContext mit der Methode "wholeTextFiles" nur den Text in das RDD.

In [ ]:
def readDataFiles(path):
    global sc
    return dataRDD

In [ ]:
path = "language-classifier-data"
deRDD = readDataFiles(path + "/de_DE")
enRDD = readDataFiles(path + "/en_UK")
esRDD = readDataFiles(path + "/es_ES")
frRDD = readDataFiles(path + "/fr_FR")
itRDD = readDataFiles(path + "/it_IT")
nlRDD = readDataFiles(path + "/nl_NL")

In [ ]:
printRDD(deRDD)

### Feature Vector erstellen

In [ ]:
charset = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'á', 'â', 'ã', 'ä', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ñ', 'ò', 'ó', 'ô', 'ö', 'ù', 'ú', 'û', 'ü']
print("size of character set", len(charset))

bigram_dict = {}
idx = 0
for c1 in charset:
    for c2 in charset:
        bigram_dict[c1+c2] = idx
        idx += 1

print("number of bigrams", len(bigram_dict))

In [ ]:
def initBigramVector():
    return np.zeros(len(bigram_dict), dtype = np.int32)

### Bigrams erstellen

In [ ]:
def getBigramVector(text):
    global bigram_dict
    bv = initBigramVector()
    
    i = 0
    while i < len(text) - 1:
        bigram = text[i].lower() + text[i+1].lower()
        if bigram in bigram_dict:
            bv[bigram_dict[bigram]] += 1
        i += 1
    
    return bv

In [ ]:
np.set_printoptions(threshold=sys.maxsize)
print(getBigramVector("Ich bin ein kurzer Text"))

### Train und Test Daten erstellen

In [ ]:
dataRDD = deRDD.map(lambda x: LabeledPoint(0.0, getBigramVector(x))) \
    .union(enRDD.map(lambda x: LabeledPoint(1.0, getBigramVector(x)))) \
    .union(esRDD.map(lambda x: LabeledPoint(2.0, getBigramVector(x)))) \
    .union(frRDD.map(lambda x: LabeledPoint(3.0, getBigramVector(x)))) \
    .union(itRDD.map(lambda x: LabeledPoint(4.0, getBigramVector(x)))) \
    .union(nlRDD.map(lambda x: LabeledPoint(5.0, getBigramVector(x))))

In [ ]:
print(dataRDD.count())

In [ ]:
print(dataRDD.first().label)
print(dataRDD.first().features)

### Übung 3

Splitte das "dataRDD" in ein Test und Train RDD mit dem Namen "trainingRDD" und "testRDD". Dies kannst du mit randomSplit machen. Das Trainset sollte 80% der Daten beinhalten und das Testset 20%.

### Klassifizieren und Bewerten

In [ ]:
model = NaiveBayes.train(trainingRDD, 1.0)

In [ ]:
predictionAndLabels = testRDD.map(lambda lp: (float(model.predict(lp.features)), lp.label))
metrics = MulticlassMetrics(predictionAndLabels)
print("Accuracy:",metrics.accuracy)

### Übung 4

Teste den Classifier. Rufe auf dem Model die Methode Predict auf. Zuerst muss der Bigram Vector des Textes ermittelt werden. Dies kannst du mit der Methode "getBigramVector" machen.